<a href="https://colab.research.google.com/github/dianatarasovets/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.3'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,846 kB]
Get:14 http://security.ubuntu.

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=3d6a0c9ee5903be7108d4e73e98a3ea015b80e852fec62173d0739b36794338c
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [9]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_data')
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import mean
from pyspark.sql.functions import round, col

four_bed_per_year = df.filter(df.bedrooms == 4)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("avg_price"))

four_bed_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2020-04-13| 303153.3|
|2020-02-26| 295492.0|
|2019-08-22| 331777.3|
|2019-08-08|305486.56|
|2021-11-03|279481.33|
|2019-08-23|273917.83|
|2020-06-24|363142.22|
|2019-09-29| 286234.2|
|2021-12-23| 263829.0|
|2019-08-31|367088.29|
|2019-04-25|240981.75|
|2020-06-08| 269630.0|
|2020-11-12|314268.22|
|2021-04-06|250681.71|
|2022-05-17|279781.33|
|2020-09-12|265752.09|
|2022-03-30| 273248.0|
|2019-01-17|333439.09|
|2021-08-30| 363022.0|
|2019-04-26|318184.93|
+----------+---------+
only showing top 20 rows



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_three_bath_per_year = df.filter((df.bedrooms == 3) & (df.bathrooms == 3)) \
    .groupBy("date") \
    .agg(round(mean("price"), 2).alias("avg_price"))
three_bed_three_bath_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-08-23| 304089.5|
|2021-11-03| 370813.8|
|2020-04-13| 298782.0|
|2020-02-26|237994.75|
|2019-08-22| 288818.5|
|2019-08-08| 380345.2|
|2021-12-23|588392.75|
|2019-04-25| 316485.6|
|2019-09-29|280435.13|
|2020-06-24| 271525.5|
|2019-08-31| 255717.0|
|2020-09-12| 449314.0|
|2020-06-08| 251318.4|
|2021-04-06|506720.67|
|2022-05-17|301401.25|
|2020-11-12| 143575.5|
|2021-10-25|329296.14|
|2020-06-22|249791.25|
|2019-04-26| 289360.5|
|2020-01-05|296966.71|
+----------+---------+
only showing top 20 rows



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_per_year = df.filter(
    (df.bedrooms == 3) &
    (df.bathrooms == 3) &
    (df.floors == 2) &
    (df.sqft_living >= 2000)
) \
    .groupBy("date") \
    .agg(round(mean("price"), 2).alias("avg_price"))
avg_price_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-11-03|420228.67|
|2020-02-26| 189833.5|
|2019-08-08| 429942.0|
|2019-08-23| 409256.0|
|2019-08-22| 176381.0|
|2020-04-13| 339996.0|
|2019-09-29| 228222.0|
|2019-08-31| 189067.0|
|2020-06-08| 185722.0|
|2020-09-12| 428180.0|
|2021-10-25| 406129.0|
|2022-03-30| 218683.5|
|2019-01-17|302354.33|
|2021-02-18| 351566.0|
|2020-06-20| 143247.0|
|2019-04-26| 146548.0|
|2020-06-22| 275864.0|
|2021-08-30| 162626.0|
|2019-06-19| 281956.5|
|2021-09-23| 369045.0|
+----------+---------+
only showing top 20 rows



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 8.559226989746094e-05 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_data")

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 6.4849853515625e-05 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_sales', mode='overwrite')

In [18]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('p_sales')

In [19]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_sales')

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 8.225440979003906e-05 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_data")

In [22]:
# 15. Check if the home_sales is no longer cached
spark.stop()
